# Formatting Output and Speaking**
**Modèle** : *Llama-3.2-1B (Instruct)* via **Ollama**  
**Objectif** : **Contrôler le format de sortie** (listes, tableaux, JSON/YAML) et le **style d’expression** (ton, concision, voix) pour obtenir des réponses exploitables.

## Pourquoi le format & le style comptent ?
- **Lisibilité** : pour un lecteur humain (listes, étapes, titres).  
- **Exploitation** : pour une machine (JSON/YAML strict).  
- **Constance** : même style/ton sur plusieurs réponses (voix de marque).

In [ ]:
# (Optionnel) Installer/mettre à jour le client Python Ollama
# !pip install -q --upgrade ollama

In [1]:
import ollama

def assert_ollama_ready():
    try:
        _ = ollama.list()
        print("✅ Ollama est accessible en local.")
    except Exception as e:
        raise SystemExit(
            "❌ Impossible de contacter Ollama. Démarrez le serveur (ex: 'ollama serve').\n"
            f"Détails: {e}"
        )

assert_ollama_ready()

try:
    print("⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...")
    ollama.pull('llama3.2:1b')
    print("✅ Modèle prêt.")
except Exception as e:
    print("ℹ️ Vérifiez le tag (ex: 'llama3.2:1b').")
    raise

✅ Ollama est accessible en local.
⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...
✅ Modèle prêt.


## Fonctions utilitaires
- `chat_fmt(system, user, temperature)` : gère un **system prompt** pour imposer ton & format.  
- `expect_json(system, user, temperature)` : vérifie que la sortie est un **JSON** valide.

In [2]:
from typing import Optional
import json

def chat_fmt(system_prompt: str, user_prompt: str, temperature: float = 0.2) -> str:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})
    r = ollama.chat(model="llama3.2:1b", messages=messages, options={"temperature": temperature})
    return r["message"]["content"]

def expect_json(system_prompt: str, user_prompt: str, temperature: float = 0.0):
    out = chat_fmt(system_prompt, user_prompt, temperature)
    print(out)
    try:
        obj = json.loads(out)
        print("\n✅ JSON valide — clés:", list(obj.keys()))
    except Exception as e:
        print("\n❌ Sortie non JSON ou invalide:", e)

## 1) Listes & étapes numérotées

In [3]:
system_list = (
    "Tu es un formateur. Réponds **uniquement** par une liste à puces (3 à 5 puces), phrases courtes."
)
user_q = "Donne des bonnes pratiques pour rédiger un prompt efficace."
print(chat_fmt(system_list, user_q))

Voici quelques bonnes pratiques pour rédiger un prompt efficace :

*   **Clé de recherche** : identifiez les mots clés et les concepts qui seront abordés dans le prompt.
*   **Objectif** : définissez l'objectif du prompt, c'est-à-dire ce que vous souhaitez obtenir ou explorer.
*   **Contenu** : résumez brièvement le contenu de votre document ou de votre projet.
*   **Format** : indiquez si vous avez besoin d'un format spécifique (par exemple, PDF, Word, etc.).
*   **Durée** : précisez la durée de l'activité ou du projet.
*   **Références** : mentionnez les sources ou les ressources qui seront utilisées dans le prompt.


In [4]:
system_steps = (
    "Tu es un coach. Réponds en **5 étapes numérotées** (format '1. ...')."
)
user_q = "Explique comment tester un prompt et l'améliorer rapidement."
print(chat_fmt(system_steps, user_q))

Voici les 5 étapes pour tester un prompt et l'améliorer rapidement :

1. **Définir le but de la modification** : Avant de commencer à modifier le prompt, identifiez clairement ce que vous voulez accomplir. Quels sont vos objectifs ? Quel est votre but principal ? Cela vous aidera à rester concentré et à prendre des décisions éclairées.

2. **Analyser la fonctionnalité** : Examine attentivement le comportement du prompt actuel. Quels sont ses besoins spécifiques ? Quelles actions souhaitez-vous que le user effectue pour répondre à son besoin ? Cela vous aidera à identifier les points faibles et les opportunités d'amélioration.

3. **Identifiez les problèmes** : Évaluez les problèmes qui existent actuellement avec le prompt. Quels sont les erreurs, les limites ou les difficultés que les utilisateurs rencontrent ? Cela vous aidera à identifier les points de faiblesse et à trouver des solutions pour les résoudre.

4. **Proposer des alternatives** : Créez des alternatives pour résoudre les 

## 2) Tableaux Markdown

In [5]:
system_table = (
    "Tu es un rédacteur technique. Réponds **uniquement** avec un tableau Markdown à 3 colonnes : "
    "| Critère | Option A | Option B | et **aucun autre texte**."
)
user_q = "Compare 'IA générative' (A) et 'IA symbolique' (B) sur 4 critères au choix."
print(chat_fmt(system_table, user_q))

| Critère | IA Générative (A) | IA Symbolique (B) |
| --- | --- | --- |
| **Complexité** | Plus complexe, nécessite des connaissances plus vives en informatique et en logiciel | Plus simple, nécessite moins de connaissances en informatique et en logiciel |
| **Applicabilité** | Plus adaptée aux tâches répétitives et régulières | Plus adaptable aux tâches complexes et à l'analyse de données |
| **Intelligence Artificielle** | Nécessite des connaissances plus vives en intelligence artificielle pour optimiser les processus | Nécessite moins de connaissances en intelligence artificielle, mais peut être utilisée pour certaines tâches |
| **Utilisation** | Typiquement utilisée dans les applications de gestion de projet et de production | Typiquement utilisée dans les applications de traitement de données et d'analyse de données |


## 3) JSON strict (machine-consommable)

In [17]:
system_json = (
    "Tu es un générateur d'outputs machine-consommables. "
    "La réponse doit être **uniquement** un objet JSON valide, sans texte autour."
)

system_json2 = (
    "Tu es un générateur d'outputs machine-consommables. "
    "Réponds avec un **objet JSON valide** UNIQUEMENT, sans Markdown, sans texte autour. "
    "Commence par { et termine par }."
)
user_q = (
    "Retourne un JSON avec: {"
    "\"concepts\":[\"prompt\",\"few_shot\"],"
    "\"definition_courte\":\"\","
    "\"niveau\":\"debutant\"}"
)
expect_json(system_json2, user_q)

{ "concepts": ["prompt", "few_shot"], "definition_courte": "", "niveau": "debutant" }

✅ JSON valide — clés: ['concepts', 'definition_courte', 'niveau']


## 4) YAML (lisible humain)

In [23]:
system_yaml = (
    "Tu es un assistant technique. Réponds **uniquement** en YAML valide."
)
user_q = (
    "Donne une ébauche de config pour une évaluation de prompt avec champs:"
    " task: nom, metrics: [clarity, faithfulness], max_tokens: 128"
)
print(chat_fmt(system_yaml, user_q))

```yml
prompt:
  type: evaluation
  description: Évaluation du prompt
  task: task
  name: évaluation-prompt
  parameters:
    - name: task_name
      type: string
      default: "évaluation-prompt"
    - name: metrics
      type: list
      items:
        - name: clarity
          type: float
        - name: faithfulness
          type: float
    - name: max_tokens
      type: integer
      default: 128
```

Ce fichier YAML définit une ébauche de configuration pour une évaluation de prompt. Il contient les champs suivants :

- `task`: défini comme étant un "task" (un type d'évaluation)
- `name`: le nom du task, qui peut être défini par l'utilisateur
- `parameters`: un tableau de paramètres pour le task
  - `task_name` : le nom du task, qui peut être défini par l'utilisateur
  - `metrics` : une liste de champs de mesure (clarity et faithfulness) qui seront utilisés pour l'évaluation
    - `clarity` : un champ de mesure float qui représente la clarté du prompt
    - `faithfulness` : un 

## 5) Ton & voix (speaking)

In [24]:
system_formel = (
    "Tu es un rédacteur formel et précis. Style neutre, sans métaphores. 80 mots max."
)
user_q = "Explique la différence entre IA générative et IA discriminative."
print(chat_fmt(system_formel, user_q))

L'IA générative et l'IA discriminative sont deux types d'applications de l'intelligence artificielle (IA) qui utilisent différentes approches pour traiter les données.

L'IA générative utilise des algorithmes pour créer des modèles ou des réseaux de neurones basés sur les données. Elle est conçue pour analyser et comprendre le comportement des données, mais elle ne prend pas de décision basée sur ces analyses.

En revanche, l'IA discriminative utilise des algorithmes pour prédire une variable ou un attribut en fonction d'une variable ou d'un attribut. Elle prend donc des décisions basées sur les analyses et les modèles générés par la génération de données.


In [25]:
system_friendly = (
    "Tu es un coach amical. Ton chaleureux, phrases courtes, émojis permis (max 2). 70 mots max."
)
user_q = "Explique la différence entre IA générative et IA discriminative."
print(chat_fmt(system_friendly, user_q))

Bonjour ! 🤗

L'IA générative et l'IA discriminative sont deux types d'applications de l'intelligence artificielle (IA) qui utilisent différentes approches pour traiter les données.

L'IA discriminative est une approche qui cherche à prédire un résultat ou une décision basée sur des caractéristiques spécifiques des données. Elle utilise souvent des modèles de réseaux neuronaux et des algorithmes de classification pour faire cela.

L'IA générative, en revanche, est une approche qui cherche à créer quelque chose (par exemple, un modèle de prévision ou un graphique) basée sur les données. Elle utilise souvent des modèles de réseaux neuronaux et des algorithmes de génération pour créer ce que l'on veut.

En somme, l'IA discriminative est comme un photographe qui prend une photo d'un objet, tandis que l'IA générative est comme un artiste qui crée une œuvre d'art basée sur les données ! 🎨📸


## 6) Contrôle de longueur & verbiage

In [27]:
system_len = (
    "Tu es concis. Réponds en **≤ 50 mots**."
)
user_q = "Donne une définition simple de 'embedding' en IA."
print(chat_fmt(system_len, user_q))


L'embedding est un processus dans l'intelligence artificielle (IA) qui consiste à représenter des données non structurées sous forme de vecteurs ou d'espaces numériques, permettant ainsi une représentation et une analyse plus efficaces.


## 7) Blocs de code & pseudo-citations

In [28]:
system_code = (
    "Tu es un instructeur. Réponds avec: 1) un bloc de code Markdown Python minimal, 2) une liste de 2 références au format [ref]."
)
user_q = "Montre un exemple d'appel à une API de LLM (pseudo-code) et ajoute 2 références générales."
print(chat_fmt(system_code, user_q))

Exemple de pseudo-code pour appeler une API de LLM en Python :
```python
import requests

def get_llm_api(url: str, api_key: str) -> dict:
    """
    Appelle l'API de LLM et récupère les résultats.
    
    Args:
        url (str): L'adresse URL de l'API.
        api_key (str): Le clé d'accès à l'API.
    
    Returns:
        dict: Les résultats de la requête.
    """
    headers = {'Authorization': f'Bearer {api_key}'}
    response = requests.get(url, headers=headers)
    return response.json()
```
Exemple de référence générale :
```python
# Récupère les informations de l'utilisateur
user_id = 12345

# Appelle l'API de LLM pour obtenir des réponses
llm_api_response = get_llm_api('https://api.llm.com/v1/users', 'YOUR_API_KEY')

# Affiche les résultats
print(llm_api_response)
```
Références générales :
```python
# Récupère les informations de l'utilisateur et affiche les résultats
def get_user_info(user_id: int, api_key: str) -> dict:
    """
    Récupère les informations d'un utilisa

## 8) Indices pour fiabiliser les formats
- Rappeler **dans `system`** : « **Uniquement** JSON/YAML/Tableau Markdown, sans texte autour ».  
- **Température basse** (`0.0–0.2`) pour formats stables.  
- Ajouter une **stratégie de fallback** : `null` si info manquante.

## Template de `system` pour format + ton
Copiez/collez et adaptez :

Tu es un <persona>. **Sortie uniquement au format <JSON/YAML/Markdown-table/puces/étapes>**.  
Respecte strictement : <structure demandée>.  
Ton : <formel/friendly/marketing/technique>.  
Longueur : <N mots max>.  
Si information manquante : indique `null`/"inconnu".

## À retenir (Phase 4)
- Le **format** (listes, tableaux, JSON/YAML) et le **style** sont pilotés par le rôle `system`.  
- Utiliser des formulations strictes (« **uniquement** JSON », « **5 étapes** », « **≤ 50 mots** »).  
- Température basse pour **stabilité** du format.
